In [3]:
import os
import sys

cwd=os.path.dirname(os.getcwd())
sys.path.append(cwd)
print(cwd)

from oligo_designer_toolsuite.oligo_specificity_filter import Blastn, Bowtie, Bowtie2, ExactMatches, BowtieSeedRegion, LigationRegionCreation, SeedRegionCreationStandard, SeedRegionCreationPercentage

from oligo_designer_toolsuite.utils._data_parser import read_oligos_DB_tsv
from oligo_designer_toolsuite.IO._database import CustomDB
from oligo_designer_toolsuite.oligo_specificity_filter._specificity_filters import SpecificityFilter

/home/francesco/Desktop/Work/oligo-designer-toolsuite


In [4]:
n_jobs=10
ligation_region=10
dir_output=cwd + "/output"
dir_annotations=dir_output +"/annotation"
dir_specificity = dir_output + "/specificity"
file_probe_info=dir_annotations + "/oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript.tsv"
file_removed_genes = dir_output + "/genes_with_insufficient_probes.txt"
print(file_probe_info)
min_probes_per_gene=2
file_transcriptome_fasta=dir_annotations + "/reference_DB_unknown_unknown_Custom_release_unknown_gene_transcript.fna"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50
seed_region_creation = LigationRegionCreation(ligation_region_size=ligation_region)


/home/francesco/Desktop/Work/oligo-designer-toolsuite/output/annotation/oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript.tsv


In [5]:
probe_info_dict=read_oligos_DB_tsv(file_probe_info)
for gene in probe_info_dict.keys():
    print(f"{gene}, {len(list(probe_info_dict[gene].keys()))}")

WASH7P, 3235
DDX11L1, 5652
TRNT, 59
NOC2L, 5736
PLEKHN1, 2913
AGRN, 5284
UBE2J2, 8190
DVL1, 2960
MIB2, 10881
LOC112268402_1, 9285


Test exact match filter

In [17]:
exact_match_filter=ExactMatches(dir_specificity)
#exact_match_filter=ExactMatches(n_jobs, dir_output, dir_annotations)

In [18]:
probes_updated = exact_match_filter.apply(probe_info_dict, file_transcriptome_fasta, 10)
#probes_updated = exact_match_filter.apply(probe_info_dict)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

WASH7P, 3235
DDX11L1, 5652
TRNT, 59
NOC2L, 5736
PLEKHN1, 2913
AGRN, 5284
UBE2J2, 8164
DVL1, 2960
MIB2, 10881
LOC112268402_1, 9285


Test Blast filter

In [19]:
blast_filter=Blastn(dir_specificity, word_size,percent_identity, coverage)

In [20]:
probes_updated = blast_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")
print(probes_updated["NOC2L"])

WASH7P, 0
DDX11L1, 0
TRNT, 0
NOC2L, 28
PLEKHN1, 454
AGRN, 483
UBE2J2, 1341
DVL1, 164
MIB2, 1010
LOC112268402_1, 0
{'NOC2L_21101': {'probe_sequence': Seq('CCGAATCCGAGTCCGAAAATTCTCCACAAG'), 'transcript_id': ['NM_015658.4'], 'exon_id': ['NM_015658.4_exon2'], 'chromosome': '1', 'start': [958984], 'end': [959014], 'strand': '-', 'length': 30, 'GC_content': 50.0, 'melt_temp': 58.4, 'melt_temp_arm1': 38.98, 'melt_temp_arm2': 40.88, 'dif_melt_temp_arms': 1.9, 'ligation_site': 13.0}, 'NOC2L_21103': {'probe_sequence': Seq('GAATCCGAGTCCGAAAATTCTCCACAAGCG'), 'transcript_id': ['NM_015658.4'], 'exon_id': ['NM_015658.4_exon2'], 'chromosome': '1', 'start': [958986], 'end': [959016], 'strand': '-', 'length': 30, 'GC_content': 50.0, 'melt_temp': 58.71, 'melt_temp_arm1': 42.65, 'melt_temp_arm2': 41.19, 'dif_melt_temp_arms': 1.46, 'ligation_site': 15.0}, 'NOC2L_21104': {'probe_sequence': Seq('AATCCGAGTCCGAAAATTCTCCACAAGCGG'), 'transcript_id': ['NM_015658.4'], 'exon_id': ['NM_015658.4_exon2'], 'chromosome'

Test the two filters one after the other

In [11]:
dir_annotation = '/home/francesco/Desktop/Work/NCBI'
annotation = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.fna'
ligation_seed_region = LigationRegionCreation(ligation_region_size=10)
seed_region_filter = BowtieSeedRegion(dir_specificity, ligation_seed_region)
db = CustomDB(25, 30, None, dir_output=dir_output, file_annotation=annotation, file_sequence = sequence)
db.read_reference_DB(file_reference_DB=file_transcriptome_fasta)
db.read_oligos_DB(file_oligos_DB_tsv=file_probe_info, format='tsv')
specificity_filters = SpecificityFilter([exact_match_filter,seed_region_filter, blast_filter])
db = specificity_filters.apply(DB=db, n_jobs=n_jobs)
for gene in db.oligos_DB.keys():
    print(f"{gene}, {len(list(db.oligos_DB[gene].keys()))}")

/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:64: UserWarning: Species not specified.
  warnings.warn("Species not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:68: UserWarning: Genome assembly not specified.
  warnings.warn(f"Genome assembly not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:72: UserWarning: Annotation release not specified.
  warnings.warn(f"Annotation release not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:76: UserWarning: Annotation source not specified.
  warnings.warn(f"Annotation source not specified.")
# reads processed: 59
# reads with at least one alignment: 59 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 59 alignments
# reads processed: 3235
# reads with at least one alignment: 3235 (100.00%)
# reads that failed 

NOC2L, 27
PLEKHN1, 372
AGRN, 483
UBE2J2, 1299
DVL1, 164
MIB2, 992


Test Bowtie filter

In [ ]:
min_mismatches=4
mismatch_region=5
ligation_region=0

bowtie_filter=Bowtie(dir_specificity, min_mismatches, mismatch_region)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
#find a probe with mismatch in ligation region and run on that. 
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")



# reads processed: 59
# reads with at least one alignment: 59 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 59 alignments
# reads processed: 2960
# reads with at least one alignment: 2960 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 4096 alignments
# reads processed: 2913
# reads with at least one alignment: 2913 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 3638 alignments
# reads processed: 3235
# reads with at least one alignment: 3235 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 30287 alignments
# reads processed: 5284
# reads with at least one alignment: 5284 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 5920 alignments
# reads processed: 5652
# reads with at least one alignment: 5652 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 55510 alignments
# reads processed: 5736
# reads with at least one alignment: 5736 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 11077 alignments
# reads processe

WASH7P, 0
DDX11L1, 0
TRNT, 59
NOC2L, 1525
PLEKHN1, 2421
AGRN, 5284
UBE2J2, 7937
DVL1, 2654
MIB2, 9760
LOC112268402_1, 0


Test Bowtie 2 filter

In [4]:
bowtie2_filter=Bowtie2(dir_specificity)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie2_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

Settings:
  Output files: "reference.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/francesco/Desktop/Work/oligo-designer-toolsuite/output/annotation/reference_DB_unknown_unknown_Custom_release_unknown_gene_transcript.fna
Reading reference sizes


Building a SMALL index


  Time reading reference sizes: 00:00:02
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 49779804
Using parameters --bmax 37334853 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 37334853 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:03
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:01
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:02
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 12 samples
  (

Traceback (most recent call last):
  File "/home/francesco/miniconda3/envs/oligo/bin/bowtie2-build", line 113, in <module>
    main()
  File "/home/francesco/miniconda3/envs/oligo/bin/bowtie2-build", line 110, in main
    sys.exit(subprocess.call(argv))
  File "/home/francesco/miniconda3/envs/oligo/lib/python3.10/subprocess.py", line 347, in call
    return p.wait(timeout=timeout)
  File "/home/francesco/miniconda3/envs/oligo/lib/python3.10/subprocess.py", line 1204, in wait
    return self._wait(timeout=timeout)
  File "/home/francesco/miniconda3/envs/oligo/lib/python3.10/subprocess.py", line 1938, in _wait
    (pid, sts) = self._try_wait(0)
  File "/home/francesco/miniconda3/envs/oligo/lib/python3.10/subprocess.py", line 1896, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
ligation_seed_region = LigationRegionCreation(ligation_region_size=10)
standard_seed_region = SeedRegionCreationStandard(start=5, end = 25)
prec_seed_region = SeedRegionCreationPercentage(start=0.2, end = 0.8)

bowtie_seed_region = BowtieSeedRegion(dir_specificity, ligation_seed_region)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie_seed_region.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

# reads processed: 3235
# reads with at least one alignment: 3235 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 34048 alignments
# reads processed: 59
# reads with at least one alignment: 59 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 59 alignments
# reads processed: 5652
# reads with at least one alignment: 5652 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 49738 alignments
# reads processed: 2913
# reads with at least one alignment: 2913 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 4040 alignments
# reads processed: 5284
# reads with at least one alignment: 5284 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 6730 alignments
# reads processed: 5736
# reads with at least one alignment: 5736 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 10151 alignments
# reads processed: 2960
# reads with at least one alignment: 2960 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 4428 alignments
# reads processe

WASH7P, 0
DDX11L1, 104
TRNT, 59
NOC2L, 3541
PLEKHN1, 2361
AGRN, 5284
UBE2J2, 7748
DVL1, 2783
MIB2, 9792
LOC112268402_1, 0
